In [1]:
import plotly.graph_objs as go  # 載入plotly套件
import numpy as np              # 載入numpy套件
import pandas as pd             # 載入pandas套件
import datetime as dt           # 載入datetime套件
from datetime import datetime
from plotly.subplots import make_subplots  #載入plotly畫子圖的套件

In [2]:
profit_df = pd.read_excel('每日損益紀錄.xlsx')
profit_df.head(5)

,日期,每日損益資金(元)
0,1998-10-26,799.984
1,1998-10-27,-19999.600
2,1998-10-28,10199.796
3,1998-10-29,13999.720
4,1998-10-30,16599.668


In [3]:
#初始化資金1000000
capital = 1000000
# 初始化策略累積資金
profit_df.loc[0, '策略累積資金(金額)'] = capital

# 開始累積
for i in range(1, len(profit_df)):
    profit_df.loc[i, '策略累積資金(金額)'] = profit_df.loc[i-1, '策略累積資金(金額)']+profit_df.loc[i, '每日損益資金(元)']

profit_df.head(5)

,日期,每日損益資金(元),策略累積資金(金額)
0,1998-10-26,799.984,1000000.000
1,1998-10-27,-19999.600,980000.400
2,1998-10-28,10199.796,990200.196
3,1998-10-29,13999.720,1004199.916
4,1998-10-30,16599.668,1020799.584


In [4]:
# 計算每日報酬率
profit_df['策略每日報酬率'] = profit_df['策略累積資金(金額)'].pct_change()
profit_df.loc[0, '策略每日報酬率'] = 0  # 初始化第一天策略每日報酬率

# 計算累積報酬率
profit_df['策略累積報酬率'] = (1 + profit_df['策略每日報酬率']).cumprod()-1
profit_df.loc[0, '策略累積報酬率'] = 0  # 初始化第一天策略累積報酬率

# 計算Drawdown
# 計算每個時間點的最大價格
profit_df['每個時點最大策略累積資金(金額)'] = profit_df['策略累積資金(金額)'].cummax()
# 計算每個時間點的 Drawdown
profit_df['Drawdown'] = (profit_df['策略累積資金(金額)'] - profit_df['每個時點最大策略累積資金(金額)']) / capital

#### 計算Max Drawdown

In [6]:
# 定義繪製策略損益圖方法
def draw_PnL(trace1, trace2,  title1, title2):
    
    # 打開子畫布
    fig = make_subplots(rows=2, cols=1, subplot_titles=(title1, title2), row_heights=[0.75, 0.25], shared_xaxes=True, vertical_spacing=0.1)
    
    # 帶入資料到子畫布的特定位置
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)
    
    # 設定子畫布共同屬性
    fig.update_layout(
        title='台指期近月指數均線交叉', # 設定圖表標題名稱
        plot_bgcolor='white', # 設定底色為白色
    )
    
    # 設定子畫布屬性
    fig.update_xaxes(row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')
    fig.update_xaxes(title_text="日期", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')

    #展示圖表
    fig.show()
    
    return

In [7]:
# 將日期轉換為 datetime 物件
profit_df['日期'] = pd.to_datetime(profit_df['日期'])

**<H>特殊格式<H>**

In [8]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '台指期月結算期貨損益', 'Drawdown')

**<H>一般格式<H>**

In [9]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '台指期月結算期貨損益', 'Drawdown')

In [10]:
# 計算績效指標
total_return = profit_df.loc[len(profit_df)-1, '策略累積報酬率']
annualized_return = (1+profit_df.loc[len(profit_df)-1, '策略累積報酬率'])**(252/len(profit_df))-1  # 年化報酬率
annualized_volatility = np.std(profit_df['策略每日報酬率'])*np.sqrt(252)  # 年化波動率
sharpe_ratio = annualized_return/annualized_volatility  # 年化夏普值
max_drawdown = profit_df['Drawdown'].min()  # 最大回撤(MDD)

print("總報酬率: {}%".format(round(total_return*100,2)))
print("年化報酬率: {}%".format(round(annualized_return*100,2)))
print("年化波動率: {}%".format(round(annualized_volatility*100,2)))
print("年化夏普值: {}".format(round(sharpe_ratio,2))) 
print("最大回撤(MDD): {}%".format(round(max_drawdown*100,2)))

總報酬率: 469.51%
年化報酬率: 11.81%
年化波動率: 20.47%
年化夏普值: 0.58
最大回撤(MDD): -73.36%
